In [ ]:
import cv2
import numpy as np
import geopandas as gpd
from shapely.geometry import Polygon
import json
import os
from skimage.feature import local_binary_pattern
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler
from skimage.color import rgb2gray
from skimage.filters import prewitt_h, prewitt_v
from skimage.feature import greycomatrix, greycoprops
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.applications.resnet import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

In [ ]:
# Load the xBD dataset images
def load_images(image_path, image_filenames):
    images = []
    for filename in image_filenames:
        image = cv2.imread(os.path.join(image_path, filename))  # Load image using OpenCV
        images.append(image)
    return images

In [ ]:
# Load the corresponding GeoJSON files and extract the building polygons
def load_gis_data(geojson_path):
    geojson_filenames = os.listdir(geojson_path)  
    polygons = []
    for filename in geojson_filenames:
        with open(os.path.join(geojson_path, filename)) as file:
            data = json.load(file)
            features = data['features']
            for feature in features:
                if 'wkt' in feature:
                    wkt_string = feature['wkt']
                    polygon = Polygon(wkt_string)
                    polygons.append(polygon)
    gis_data = gpd.GeoDataFrame(geometry=polygons)
    return gis_data

In [ ]:
# Preprocess the image: convert to grayscale and resize
def preprocess_image(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized_image = cv2.resize(gray_image, (224, 224))
    return resized_image

In [ ]:
# Extract texture features using Local Binary Patterns (LBP)
def extract_texture_features(image):
    lbp_image = local_binary_pattern(image, P=8, R=1, method='uniform')
    hist, _ = np.histogram(lbp_image.ravel(), bins=np.arange(0, 10), density=True)
    return hist

In [ ]:
# Extract edge features using Prewitt filters
def extract_edge_features(image):
    edges_h = prewitt_h(image)
    edges_v = prewitt_v(image)
    edge_features = np.hstack((edges_h.ravel(), edges_v.ravel()))
    return edge_features

In [ ]:
# Extract color features using GLCM
def extract_color_features(image):
    gray_image = rgb2gray(image)
    glcm = greycomatrix((gray_image * 255).astype(np.uint8), distances=[5], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = greycoprops(glcm, 'contrast')[0, 0]
    homogeneity = greycoprops(glcm, 'homogeneity')[0, 0]
    energy = greycoprops(glcm, 'energy')[0, 0]
    correlation = greycoprops(glcm, 'correlation')[0, 0]
    color_features = np.array([contrast, homogeneity, energy, correlation])
    return color_features

In [ ]:
# Check if an image overlaps with any damaged area in the GIS data
def image_overlaps_with_damaged_area(image_filename, gis_data):
    geojson_filename = image_filename.replace('_post_', '_pre_').replace('.png', '.json')
    geojson_filepath = os.path.join(geojson_path, geojson_filename)
    with open(geojson_filepath) as file:
        data = json.load(file)
        features = data['features']
        for feature in features:
            if 'wkt' in feature:
                wkt_string = feature['wkt']
                polygon = Polygon(wkt_string)
                if polygon.intersects(gis_data.geometry):
                    return True
    return False

In [ ]:
# Specify the paths to the xBD dataset images and corresponding GeoJSON files
image_path = 'train/images/'
geojson_path = 'train/labels/'

# Load the images and GIS data
image_filenames = os.listdir(image_path)
images = load_images(image_path, image_filenames)
gis_data = load_gis_data(geojson_path)

# Extract features and labels for each image
features = []
labels = []
for image, filename in zip(images, image_filenames):
    preprocessed_image = preprocess_image(image)
    texture_features = extract_texture_features(preprocessed_image)
    edge_features = extract_edge_features(preprocessed_image)
    color_features = extract_color_features(image)
    feature_vector = np.concatenate((texture_features, edge_features, color_features))
    features.append(feature_vector)
    if "_post_" in filename:
        labels.append(1)  # Damaged area
    else:
        labels.append(0)  # Not damaged area

In [ ]:
# Convert features and labels to NumPy arrays
X = np.array(features)
y = np.array(labels)

In [ ]:
# Check the class distribution
class_counts = np.bincount(y)
if class_counts.shape[0] < 2:
    raise ValueError("The dataset should have at least two classes.")

In [ ]:
# Perform data balancing using RandomOverSampler if necessary
if np.min(class_counts) < 2:
    oversampler = RandomOverSampler(random_state=42)
    X_resampled, y_resampled = oversampler.fit_resample(X, y)
else:
    X_resampled, y_resampled = X, y

In [ ]:
# Update the feature and label arrays with the resampled data
X = X_resampled
y = y_resampled

In [ ]:
# Scale the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Reshape the feature array for use with CNN
num_samples, feature_shape = X.shape
X = X.reshape(num_samples, feature_shape, 1, 1)

In [ ]:
# Convert labels to one-hot encoding
y = to_categorical(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create the ResNet50 model
input_shape = (16, 16, 1)
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
# Evaluate the model
_, accuracy = model.evaluate(X_test, y_test)
print("Accuracy:", accuracy)